# Exploratory Data Analysis

The objective of this notebook is to select features for electricity 
price forecasting models by using historical data to:
1. Visualise trends and periodic behaviour
2. Investigate relationships between variables
3. Compare forecasts with actual historical data

## 1. Identify trends and periodic behaviour

### 1.1 Load datasets

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

plt.style.use("seaborn-v0_8-colorblind")

base_dir = os.path.join(os.path.dirname(os.getcwd()), "data", "processed")
df_electricity = pd.read_csv(
    os.path.join(base_dir, "electricity_data.csv"),
    parse_dates=["startTime", "settlementDate"],
)
df_weather = pd.read_csv(
    os.path.join(base_dir, "weather_data.csv"), parse_dates=["ob_time"]
)

# Sort by timestamps
df_electricity = df_electricity.sort_values("startTime")
df_weather = df_weather.sort_values("ob_time")

print(
    f"Electricity data memory usage: {np.sum(df_electricity.memory_usage()) / 10**6} MB"
)
print(
    f"Weather data memory usage: {np.sum(df_weather.memory_usage()) / 10**6} MB"
)

print("\nElectricity data columns:\n", df_electricity.columns)
print("\nWeather data columns:\n", df_weather.columns)

In [ ]:
df_electricity.head()

In [ ]:
df_weather.head()

### 1.2 Electricity prices

Plot a seasonal decomposition of daily mean market index prices:

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

daily_mip = df_electricity.groupby("settlementDate")["marketIndexPrice"].mean()
decomposition = seasonal_decompose(daily_mip, model="additive", period=365)

fig, axes = plt.subplots(
    4, 1, figsize=(16 / 2, 9 / 2), layout="constrained", sharex=True
)
colors = sns.color_palette("colorblind", 4)
axes[0].plot(
    decomposition.observed.index,
    decomposition.observed.values,
    label="Observed",
    color=colors[0],
)
axes[1].plot(
    decomposition.trend.index,
    decomposition.trend.values,
    label="Trend",
    color=colors[1],
)
axes[2].plot(
    decomposition.seasonal.index,
    decomposition.seasonal.values,
    label="Seasonal",
    color=colors[2],
)
axes[3].plot(
    decomposition.resid.index,
    decomposition.resid.values,
    label="Residual",
    color=colors[3],
)
axes[3].set_xlabel("Date", fontsize=10)
for ax in axes:
    ax.set_ylabel("£ / MWh", fontsize=10)
    ax.legend(loc="upper left", fontsize=8)
fig.suptitle(
    "Seasonal decomposition of daily mean electricity price", fontsize=10
)
plt.xticks(rotation=45)
plt.show()

*Comments:*

There are clear seasonal fluctuations, with lower prices during the 
summers and higher prices during the winters. 
In addition, we see a peak in the price trend, which might be attributed 
to higher natural gas prices around the same period (we will confirm this later).

Plot price histogram:

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), layout="constrained")
ax.hist(df_electricity["marketIndexPrice"], bins=100)
ax.set_xlabel("Price (£/MWh)", fontsize=10)
ax.set_ylabel("Frequency", fontsize=10)
plt.show()

*Comments:*

Unsure what type of distribution this is. In any case, it is positively skewed.

Let's investigate monthly, weekly, daily and intraday price patterns:

In [ ]:
# Box plot by month (not showing outlier points)
fig, ax = plt.subplots(figsize=(4, 3), layout="constrained")
df_electricity["month"] = df_electricity["settlementDate"].dt.month
df_electricity.boxplot(column="marketIndexPrice", by="month", ax=ax, sym="")
ax.set_xlabel("Month", fontsize=10)
ax.set_ylabel("Price (£/MWh)", fontsize=10)
fig.suptitle("")
plt.show()

# Average price by settlement period
sp_avg = df_electricity.groupby(df_electricity["settlementPeriod"])[
    "marketIndexPrice"
].mean()
fig, ax = plt.subplots(figsize=(16 / 4, 9 / 4), layout="constrained")
ax.plot(sp_avg.index, sp_avg.values)
ax.set_xlabel("Settlement period", fontsize=10)
ax.set_ylabel("Price (£/MWh)", fontsize=10)
ax.set_title("Mean MIP per settlement period", fontsize=10)
plt.show()

# Average price by weekday
df_electricity["day_of_week"] = df_electricity["settlementDate"].dt.dayofweek
df_electricity["is_weekend"] = df_electricity["day_of_week"] >= 5
fig, ax = plt.subplots(figsize=(16 / 4, 9 / 4), layout="constrained")
for weekend, group in df_electricity.groupby("is_weekend"):
    hourly = group.groupby(group["settlementPeriod"])[
        "marketIndexPrice"
    ].mean()
    label = "Weekend" if weekend else "Weekday"
    ax.plot(hourly.index, hourly.values, label=label)
ax.legend(loc="best", fontsize=8)
ax.set_xlabel("Settlement period", fontsize=10)
ax.set_ylabel("Price (£/MWh)", fontsize=10)
ax.set_title("Mean MIP per settlement period", fontsize=10)
plt.show()

*Comments:*
 
There are generally two peaks and two troughs in price per day. Each settlement period is 30 minutes, so the peaks are typically during the late morning hours and evening, while troughs are typically during the early morning hours and early afternoon. Weekends are cheaper than weekdays and have a much smaller morning peak than the weekdays.

Plot rolling mean and standard deviation:

In [ ]:
mip = df_electricity["marketIndexPrice"]
mip_rolling_avg = mip.rolling(48 * 7).mean().dropna()
mip_rolling_std = mip.rolling(48 * 7).std().dropna()

fig, ax = plt.subplots(figsize=(16 / 3, 9 / 3), layout="constrained")
ax.plot(mip_rolling_avg, label="Rolling Mean")
ax.plot(mip_rolling_std, label="Rolling Std. Dev.")
ax.set_xlabel("Sample")
ax.set_ylabel("£ / MWh", fontsize=10)
ax.legend(fontsize=8)
fig.suptitle("Market Index Price", fontsize=10)
plt.xticks(rotation=45)
plt.show()

### 1.3 Natural gas prices

In [ ]:
# drop duplicates
gas_prices = df_electricity.groupby(
    df_electricity["settlementDate"].dt.date
).first()["naturalGasPrice"]

fig, ax = plt.subplots(figsize=(16 / 3, 9 / 3), layout="constrained")
ax.plot(gas_prices.index, gas_prices.values, label="Natural Gas")
ax.plot(daily_mip.index, daily_mip.values, label="Market Index")
ax.set_xlabel("Date", fontsize=10)
ax.set_ylabel("Price (£ / KWh)", fontsize=10)
ax.legend()
plt.xticks(rotation=45)
plt.show()

*Comments:*

Natural gas prices appear to be strongly correlated with electricity prices.

### 1.4 Electricity generation

Let's break down the electricity generation sources:

In [ ]:
from matplotlib.dates import DateFormatter
import datetime

generation_types = [
    "NUCLEAR",
    "GAS",
    "WIND",
    "INTER",
    "BIOMASS",
    "SOLAR",
    "COAL",
    "OIL",
    "OTHER",
]

# Group by year-month and take mean
df_electricity["year_month"] = df_electricity["settlementDate"].dt.to_period(
    "M"
)
monthly_generation_by_type = df_electricity.groupby("year_month")[
    generation_types
].mean()

# Convert period index back to datetime for plotting
monthly_generation_by_type.index = (
    monthly_generation_by_type.index.to_timestamp()
)

fig, ax = plt.subplots(figsize=(16 / 2, 9 / 2), layout="constrained")
ax.set_title("Monthly mean electricity generation", fontsize=10)
colors = sns.color_palette("colorblind", len(generation_types))
xdata = monthly_generation_by_type.index
ydata = [monthly_generation_by_type[gt] / 1000 for gt in generation_types]
ax.stackplot(xdata, ydata, labels=generation_types, colors=colors)
ax.legend(fontsize=8, loc="best")
ax.xaxis.set_major_formatter(
    DateFormatter("%Y-%m")
)  # Changed format to show year-month
ax.set_xlabel("Date", fontsize=10)
ax.set_ylabel("Power (GW)", fontsize=10)
plt.xticks(rotation=45)
plt.show()

*Comments:*

Nuclear, gas and wind make up the majority of the UK's power generation.
There is some anomalous data around the end of 2022.

In [ ]:
# compute total generation
generation_by_type = df_electricity[generation_types]
df_electricity["generationTotal"] = generation_by_type.sum(axis=1)

# take 7-day rolling average
cols = generation_types + ["generationTotal"]
daily_generation = df_electricity.groupby("settlementDate")[cols].mean()
rolling_generation = daily_generation.rolling(window=7).mean()

# plot
fig, ax = plt.subplots(figsize=(16 / 3, 9 / 3), layout="constrained")
ax.plot(
    rolling_generation["generationTotal"].index,
    rolling_generation["generationTotal"].values / 1000,
    label="All sources",
)
ax.plot(
    rolling_generation["WIND"].index,
    rolling_generation["WIND"].values / 1000,
    label="Wind",
)
ax.plot(
    rolling_generation["SOLAR"].index,
    rolling_generation["SOLAR"].values / 1000,
    label="Solar",
)
ax.plot(
    rolling_generation["GAS"].index,
    rolling_generation["GAS"].values / 1000,
    label="Gas",
)
ax.plot(
    rolling_generation["NUCLEAR"].index,
    rolling_generation["NUCLEAR"].values / 1000,
    label="Nuclear",
)
ax.set_xlabel("Date", fontsize=10)
ax.set_ylabel("Power (GW)", fontsize=10)
ax.legend(fontsize=8, loc="best")
ax.set_title("Monthly mean electricity generation", fontsize=10)
plt.xticks(rotation=45)
plt.show()

Plot electricity generation breakdown for the first week in October 2023:

In [ ]:
# select smaller date range for plotting
start_date = pd.to_datetime("2023-10-01")
end_date = pd.to_datetime("2023-10-07")
mask = (df_electricity["settlementDate"] >= start_date) & (
    df_electricity["settlementDate"] <= end_date
)
df_elec_filtered = df_electricity.loc[mask].copy()

time = df_elec_filtered["startTime"]
generation_by_type = df_elec_filtered[generation_types]

fig, ax = plt.subplots(figsize=(16 / 2, 9 / 2), layout="constrained")
colors = sns.color_palette("colorblind", len(generation_types))
ax.stackplot(
    time,
    [generation_by_type[gt] / 1000 for gt in generation_types],
    labels=generation_types,
    colors=colors,
)
ax.legend(fontsize=8, loc="upper right")
ax.xaxis.set_major_formatter(DateFormatter("%Y-%m-%d"))
ax.set_xlabel("Date", fontsize=10)
ax.set_ylabel("Power (GW)", fontsize=10)
ax.set_title("October 2023 snapshot", fontsize=10)
plt.xticks(rotation=45)
plt.show()

Let's plot MIP, NGP and generation over a shorter time period (first week in October 2023):

In [ ]:
# select smaller date range
start_date = pd.to_datetime("2023-10-01")
end_date = pd.to_datetime("2023-10-07")
mask = (df_electricity["settlementDate"] >= start_date) & (
    df_electricity["settlementDate"] <= end_date
)
df = df_electricity.loc[mask]
df = df.set_index("startTime")
MIP = df["marketIndexPrice"]
NGP = df["naturalGasPrice"]
generation = df["generationTotal"] / 1000  # rescale to GW
wind = df["WIND"] / 1000  # rescale to GW
solar = df["SOLAR"] / 1000  # rescale to GW
gas = df["GAS"] / 1000  # rescale to GW
nuclear = df["NUCLEAR"] / 1000  # rescale to GW
biomass = df["BIOMASS"] / 1000  # rescale to GW

colors = sns.color_palette("colorblind", 7)

# Plot 1: Prices
fig, (ax1, ax2) = plt.subplots(
    2, 1, figsize=(16 / 2, 9 / 2), layout="constrained", sharex=True
)
ax1.plot(MIP.index, MIP, label="Market index price")
ax1.plot(NGP.index, NGP, label="Natural gas price")
# ax.set_xlabel('Date', fontsize=10)
ax1.set_ylabel("Price (£ / MWh)", fontsize=10)
ax1.legend(fontsize=8, loc="best")

# Plot 2: Generation
ax2.plot(generation.index, generation, label="All sources")
ax2.plot(wind.index, wind, label="Wind")
ax2.plot(solar.index, solar, label="Solar")
ax2.plot(gas.index, gas, label="Gas")
ax2.plot(nuclear.index, nuclear, label="Nuclear")
ax2.plot(biomass.index, biomass, label="Biomass")
ax2.set_xlabel("Date", fontsize=10)
ax2.set_ylabel("Power (GW)", fontsize=10)
ax2.legend(fontsize=8, loc="best")
fig.suptitle("October 2023 snapshot", fontsize=10)
plt.xticks(rotation=45)
plt.show()

*Comments:*

- Nuclear, gas and wind make up the majority of the energy generation
- Nuclear generation is very stable
- There are large, daily fluctuations in energy generation
- Energy from interconnectors is mostly positive, indicating that 
    imports are larger than exports
- Solar generation coincides with early peak hours
- Oil generation is neglibible
- There are clearly seasonal patterns in both wind and solar generation 
    (more solar during summers, more wind during winters).
- There are some anomalies in the generation data around the end of 2022, 
    spring 2023 and the beginning of 2024 that indicate incorrect 
    measurement or missing values for non-renewable energy sources.

In [ ]:
# drop oil generation from data
df_electricity.drop(columns="OIL", inplace=True)
generation_types.remove("OIL")

### 1.5 Weather patterns

In [ ]:
df_weather.columns

In [ ]:
# Average over locations for each timestamp
df_avg_locations = (
    df_weather.groupby("ob_time").mean(numeric_only=True).reset_index()
)

# Average over locations and dates
df_avg_locations_dates = (
    df_weather.groupby(df_weather["ob_time"].dt.date)
    .mean(numeric_only=True)
    .reset_index()
)
df_avg_locations_dates.rename(columns={"ob_time": "date"}, inplace=True)
df_avg_locations_dates.set_index("date", inplace=True)

Plot seasonal decomposition of air temperature, wind speed and 
irradiation averaged over the locations and days:

In [ ]:
# Decompose air temperature
daily_temp = df_avg_locations_dates["air_temperature"]
decomposition = seasonal_decompose(daily_temp, model="additive", period=365)

# Plot air temperature
fig, axes = plt.subplots(
    4, 1, figsize=(16 / 3, 9 / 3), layout="constrained", sharex=True
)
colors = sns.color_palette("colorblind", 4)
axes[0].plot(
    decomposition.observed.index,
    decomposition.observed.values,
    label="Observed",
    color=colors[0],
)
axes[1].plot(
    decomposition.trend.index,
    decomposition.trend.values,
    label="Trend",
    color=colors[1],
)
axes[2].plot(
    decomposition.seasonal.index,
    decomposition.seasonal.values,
    label="Seasonal",
    color=colors[2],
)
axes[3].plot(
    decomposition.resid.index,
    decomposition.resid.values,
    label="Residual",
    color=colors[3],
)
axes[3].set_xlabel("Date", fontsize=10)
for ax in axes:
    ax.set_ylabel("$\\degree \\mathrm{C}$", fontsize=10)
    ax.legend(loc="upper left", fontsize=8)
fig.suptitle(
    "Seasonal decomposition of daily mean air temperature", fontsize=10
)
plt.xticks(rotation=45)
plt.show()

# Decompose wind speed
daily_wind = df_avg_locations_dates["wind_speed"]
decomposition = seasonal_decompose(daily_wind, model="additive", period=365)

# Plot wind speed
fig, axes = plt.subplots(
    4, 1, figsize=(16 / 3, 9 / 3), layout="constrained", sharex=True
)
colors = sns.color_palette("colorblind", 4)
axes[0].plot(
    decomposition.observed.index,
    decomposition.observed.values,
    label="Observed",
    color=colors[0],
)
axes[1].plot(
    decomposition.trend.index,
    decomposition.trend.values,
    label="Trend",
    color=colors[1],
)
axes[2].plot(
    decomposition.seasonal.index,
    decomposition.seasonal.values,
    label="Seasonal",
    color=colors[2],
)
axes[3].plot(
    decomposition.resid.index,
    decomposition.resid.values,
    label="Residual",
    color=colors[3],
)
axes[3].set_xlabel("Date", fontsize=10)
for ax in axes:
    ax.set_ylabel("m/s", fontsize=10)
    ax.legend(loc="upper left", fontsize=8)
fig.suptitle("Seasonal decomposition of daily mean wind speed", fontsize=10)
plt.xticks(rotation=45)
plt.show()

# Decompose solar irradiation
daily_irrad = df_avg_locations_dates["glbl_irad_amt"]
decomposition = seasonal_decompose(daily_irrad, model="additive", period=365)

fig, axes = plt.subplots(
    4, 1, figsize=(16 / 3, 9 / 3), layout="constrained", sharex=True
)
colors = sns.color_palette("colorblind", 4)
axes[0].plot(
    decomposition.observed.index,
    decomposition.observed.values,
    label="Observed",
    color=colors[0],
)
axes[1].plot(
    decomposition.trend.index,
    decomposition.trend.values,
    label="Trend",
    color=colors[1],
)
axes[2].plot(
    decomposition.seasonal.index,
    decomposition.seasonal.values,
    label="Seasonal",
    color=colors[2],
)
axes[3].plot(
    decomposition.resid.index,
    decomposition.resid.values,
    label="Residual",
    color=colors[3],
)
axes[3].set_xlabel("Date", fontsize=10)
for ax in axes:
    ax.set_ylabel("$\\mathrm{KJ/m}^2$", fontsize=10)
    ax.legend(loc="upper left", fontsize=8)
fig.suptitle(
    "Seasonal decomposition of daily mean global solar irradiation",
    fontsize=10,
)
plt.xticks(rotation=45)
plt.show()

## 2. Relationships between variables

### 2.1 Spearman and Pearson correlations between variables

Merge dataframes:

In [ ]:
# Merge dataframes:
df_electricity["startTime"] = pd.to_datetime(
    df_electricity["startTime"]
).dt.tz_localize(None)
df_avg_locations["ob_time"] = pd.to_datetime(
    df_avg_locations["ob_time"]
).dt.tz_localize(None)
df_tmp = df_avg_locations.rename(columns={"ob_time": "startTime"})
df_merged = df_electricity.merge(df_tmp, on="startTime", how="inner")  # merge

Plot relationships between variables and electricity price:

In [ ]:
market_variables = {
    "marketIndexTradingVolume": "Market Index Trading Volume",
    "naturalGasPrice": "Natural Gas Price (£ / MWh)",
    "INDO": "National demand (MW)",
    "ITSO": "Transmission system demand (MW)",
}

generation_variables = {
    "generationTotal": "Total Generation (MW)",
    "WIND": "Wind Generation (MW)",
    "SOLAR": "Solar Generation (MW)",
    "GAS": "Gas Generation (MW)",
    "NUCLEAR": "Nuclear Generation (MW)",
    "BIOMASS": "Biomass Generation (MW)",
    "COAL": "Coal Generation (MW)",
    "INTER": "Interconnector Flow (MW)",
}

weather_variables = {
    "glbl_irad_amt": "Solar Irradiation (KJ/m^2)",
    "visibility": "Visibility (decametres)",
    "wind_speed": "Wind Speed (m/s)",
    "air_temperature": "Air Temperature (degC)",
}

variables = market_variables | generation_variables | weather_variables

variables = {
    # market variables
    "marketIndexTradingVolume": "Market Index Trading Volume",
    "naturalGasPrice": "Natural Gas Price",
    "INDO": "National demand",
    "ITSO": "Transmission system demand",
    # generation variables
    "generationTotal": "Total Generation",
    "WIND": "Wind Generation",
    "SOLAR": "Solar Generation",
    "GAS": "Gas Generation",
    "NUCLEAR": "Nuclear Generation",
    "BIOMASS": "Biomass Generation",
    "COAL": "Coal Generation",
    "INTER": "Interconnector Flow",
    # weather variables
    "glbl_irad_amt": "Solar Irradiation",
    "visibility": "Visibility",
    "wind_speed": "Wind Speed",
    "air_temperature": "Air Temperature",
}

target = "marketIndexPrice"
target_string = "MIP (£ / MIP)"

fig, axes = plt.subplots(2, 2, layout="tight", figsize=(8, 6))
for ax, (var, label) in zip(axes.flat, market_variables.items()):
    ax.scatter(df_merged[var], df_merged[target], alpha=0.3, s=1)
    ax.set_xlabel(label)
    ax.set_ylabel(target_string, fontsize=10)
plt.show()

fig, axes = plt.subplots(4, 2, layout="tight", figsize=(8, 12))
for ax, (var, label) in zip(axes.flat, generation_variables.items()):
    ax.scatter(df_merged[var], df_merged[target], alpha=0.3, s=1)
    ax.set_xlabel(label)
    ax.set_ylabel(target_string, fontsize=10)
plt.show()

fig, axes = plt.subplots(2, 2, layout="tight", figsize=(8, 6))
for ax, (var, label) in zip(axes.flat, weather_variables.items()):
    ax.scatter(df_merged[var], df_merged[target], alpha=0.3, s=1)
    ax.set_xlabel(label)
    ax.set_ylabel(target_string, fontsize=10)
plt.show()

Compute correlations between different variables:

In [ ]:
import scipy.stats

correlation_results = []
for var, label in variables.items():
    valid_data = df_merged[[var, target]].dropna()  # drop NaNs
    pearson_r, pearson_p = scipy.stats.pearsonr(
        valid_data[var], valid_data[target]
    )
    spearman_r, spearman_p = scipy.stats.spearmanr(
        valid_data[var], valid_data[target]
    )

    correlation_results.append(
        {
            "Variable": label,
            "Pearson": pearson_r,
            "Pearson p-value": pearson_p,
            "Spearman": spearman_r,
            "Spearman p-value": spearman_p,
        }
    )

corr_df = pd.DataFrame(correlation_results)
corr_df = corr_df.sort_values("Pearson", key=abs, ascending=False)

print("Correlations with Market Index Price:")
print(corr_df.to_string(index=False))

Visualise correlations:

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5), layout="constrained")
x = np.arange(len(corr_df))
width = 0.35
bars1 = ax.barh(
    x - width / 2, corr_df["Pearson"], width, label="Pearson", alpha=0.8
)
bars2 = ax.barh(
    x + width / 2, corr_df["Spearman"], width, label="Spearman", alpha=0.8
)
ax.set_yticks(x)
ax.set_yticklabels(corr_df["Variable"], fontsize=9)
ax.set_xlabel("Correlation Coefficient", fontsize=10)
ax.set_title("Pearson and Spearman correlations", fontsize=10)
ax.legend(fontsize=9)
ax.axvline(x=0, color="black", linewidth=0.8, linestyle="-", alpha=0.3)
ax.grid(axis="x", alpha=0.3)

plt.show()

### 2.2 Time-lagged correlations

Autocorrelation of market index prices:

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

mip = df_electricity["marketIndexPrice"]
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16 / 2, 9 / 3), layout="tight")
plot_acf(mip, lags=60, ax=ax1)
ax1.set_title("Price ACF", fontsize=10)
ax1.set_xlabel("Lag (half-hour intervals)", fontsize=10)
plot_pacf(mip, lags=24, ax=ax2)
ax2.set_title("Price PACF", fontsize=10)
ax2.set_xlabel("Lag (half-hour intervals)", fontsize=10)
plt.show()

The ACF function does not decay much. Let's try to difference it and see what the effect is:

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

mip = df_electricity["marketIndexPrice"]
fig, ax = plt.subplots(figsize=(16 / 3, 9 / 3), layout="tight")
plot_acf(mip.diff(1).dropna(), lags=50, ax=ax)
ax.set_title("Differenced ACF", fontsize=10)
ax.set_xlabel("Lag (half-hour intervals)", fontsize=10)
plt.show()

fig, ax = plt.subplots(figsize=(16 / 3, 9 / 3), layout="tight")
plot_pacf(mip.diff(1).dropna(), lags=50, ax=ax)
ax.set_title("Differenced PACF", fontsize=10)
ax.set_xlabel("Lag (half-hour intervals)", fontsize=10)
plt.show()

Autocorrelation of daily mean market index prices:

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

daily_prices = df_electricity.groupby("settlementDate")[
    "marketIndexPrice"
].mean()

fig, axes = plt.subplots(1, 2, figsize=(16 / 2, 9 / 4), layout="tight")
plot_acf(daily_prices, lags=30, ax=axes[0])
plot_pacf(daily_prices, lags=30, ax=axes[1])
axes[0].set_title("Daily mean price ACF", fontsize=10)
axes[1].set_title("Daily mean price PACF", fontsize=10)
axes[0].set_xlabel("Lag (days)", fontsize=10)
axes[1].set_xlabel("Lag (days)", fontsize=10)
plt.show()

Time-lagged Pearson correlation with gas prices and demand:

In [ ]:
variables = ["naturalGasPrice", "ITSO"]
max_lag = 48  # 48 settlement periods = 24 hours
lags = range(0, max_lag + 1)
results = {var: [] for var in variables}

for var in variables:
    for lag in lags:
        valid_data = df_merged[[var, target]].copy()
        valid_data[f"{var}_lagged"] = valid_data[var].shift(lag)
        valid_data = valid_data.dropna()
        if len(valid_data) > 0:
            corr, _ = scipy.stats.pearsonr(
                valid_data[f"{var}_lagged"], valid_data[target]
            )
            results[var].append(corr)
        else:
            results[var].append(np.nan)

# Plot results
fig, axes = plt.subplots(1, 2, figsize=(8, 3), layout="constrained")
axes = axes.flatten()
labels = {
    "naturalGasPrice": "Natural Gas Price (£ / MWh)",
    "ITSO": "Transmission system demand (MW)",
}

for idx, var in enumerate(variables):
    ax = axes[idx]
    ax.plot(lags, results[var], marker="o", markersize=3)
    ax.set_ylabel("Correlation", fontsize=9)
    ax.set_title(labels[var], fontsize=10)
    ax.grid(alpha=0.3)
    ax.axhline(y=0, color="black", linewidth=0.8, linestyle="--", alpha=0.3)

    # Mark maximum correlation
    max_corr_idx = np.argmax(np.abs(results[var]))
    max_corr = results[var][max_corr_idx]
    ax.plot(
        max_corr_idx,
        max_corr,
        "ro",
        markersize=6,
        label=f"Max at lag {max_corr_idx}",
    )
    ax.legend(fontsize=8)

axes[0].set_xlabel("Lag (half-hour periods)", fontsize=10)
axes[1].set_xlabel("Lag (half-hour periods)", fontsize=10)
fig.suptitle("Time-lagged Correlations with MIP", fontsize=10)
plt.show()

*Comments:* 

The second plot shows that past demand patterns are strongly correlated 
with current electricity prices with a periodic delay of half a day. 
This is intuitive--days/weather (and therefore human activity and 
electricity demand) is periodic on similar time scales. 

## 3. Forecast Data

Load forecast data:

In [ ]:
import os

path = os.path.join(
    os.path.dirname(os.getcwd()), "data", "processed", "forecast_data.csv"
)
df_forecast = pd.read_csv(path, parse_dates=["SETTLEMENT_DATE"])
df_forecast.head()

Plot demand snapshot:

In [ ]:
df_actual = df_electricity.copy()

# Get snapshot
start_date = pd.Timestamp("2023-10-01")
end_date = pd.Timestamp("2023-10-07")
mask_actual = (df_actual["settlementDate"] >= start_date) & (
    df_actual["settlementDate"] <= end_date
)
mask_forecast = (df_forecast["SETTLEMENT_DATE"] >= start_date) & (
    df_forecast["SETTLEMENT_DATE"] <= end_date
)
df_actual_snap = df_actual[mask_actual].copy()
df_forecast_snap = df_forecast[mask_forecast].copy()

# Create datetime from settlement date and period
# Settlement periods are 1-48 for half-hourly intervals
df_actual_snap["datetime"] = df_actual_snap[
    "settlementDate"
] + pd.to_timedelta((df_actual_snap["settlementPeriod"] - 1) * 30, unit="m")
df_forecast_snap["datetime"] = df_forecast_snap[
    "SETTLEMENT_DATE"
] + pd.to_timedelta((df_forecast_snap["SETTLEMENT_PERIOD"] - 1) * 30, unit="m")

# Plot
fig, ax = plt.subplots(figsize=(16 / 3, 9 / 3), layout="constrained")
ax.plot(
    df_actual_snap["datetime"],
    df_actual_snap["ITSO"] / 1000,
    label="Actual ITSO",
)
ax.plot(
    df_actual_snap["datetime"],
    df_actual_snap["INDO"] / 1000,
    label="Actual INDO",
)
ax.plot(
    df_forecast_snap["datetime"],
    df_forecast_snap["DEMAND_FORECAST"] / 1000,
    label="Demand forecast",
)
ax.legend(fontsize=8)
plt.xticks(rotation=45)
ax.set_ylabel("Power (GW)", fontsize=10)
ax.set_xlabel("Date", fontsize=10)
plt.show()

*Comments:*

The demand forecast appears to follow the actual initial national demand outturn (INDO) reasonably well.

Plot generation snapshot:

In [ ]:
wind_total_forecast = (
    df_forecast_snap["WIND_FORECAST"]
    + df_forecast_snap["EMBEDDED_WIND_FORECAST"]
)
wind_metered_forecast = df_forecast_snap["WIND_FORECAST"]
solar_forecast = df_forecast_snap["EMBEDDED_SOLAR_FORECAST"]

fig, ax = plt.subplots(figsize=(16 / 2, 9 / 2), layout="constrained")
ax.plot(
    df_actual_snap["datetime"],
    df_actual_snap["WIND"] / 1000,
    label="Wind (actual)",
)
ax.plot(
    df_actual_snap["datetime"],
    df_actual_snap["SOLAR"] / 1000,
    label="Solar (actual)",
)
ax.plot(
    df_forecast_snap["datetime"],
    wind_metered_forecast / 1000,
    ls="--",
    label="Metered wind (forec.)",
)
ax.plot(
    df_forecast_snap["datetime"],
    wind_total_forecast / 1000,
    ls="--",
    label="Metered + emb. wind (forec.)",
)
ax.plot(
    df_forecast_snap["datetime"],
    solar_forecast / 1000,
    ls="--",
    label="Emb. solar (forec.)",
)
ax.legend(fontsize=8)
plt.xticks(rotation=45)
ax.set_ylabel("Power (GW)", fontsize=10)
ax.set_xlabel("Date", fontsize=10)
plt.show()

*Comments:*

This suggests that the embedded wind generation is not included directly
in the electricity generation data and is instead an indirect effect in
demand suppression. Solar generation is accurately predicted while wind
forecasts have lower accuracy.